<a href="https://colab.research.google.com/github/jotix16/Machine-Learning/blob/master/Lab2/Svm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
print("hello")

hello
